# 1. Extract topics and short desc

In [18]:
!pip install openai langchain 


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
# LangChain basics
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain



# Langchain Loaders:
from langchain.document_loaders import YoutubeLoader

# Vector Store and retrievals
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
#import pinecone

# Chat Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# Supporting libraries
import os
from dotenv import load_dotenv

load_dotenv()

True

In [20]:

llm3 = ChatOpenAI(temperature=0,
                  model_name="gpt-3.5-turbo-0613",
                  request_timeout = 180
                )

#llm3= OpenAI(model_name="gpt-3.5-turbo-0613", )

In [21]:
!pip install youtube-transcript-api pytube


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
!pip install tiktoken


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
!pip install pytube


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from urllib.error import HTTPError

In [25]:
from pytube import YouTube

In [26]:
!python -m pip install git+https://github.com/Zeecka/pytube@fix_1060

  Cloning https://github.com/Zeecka/pytube (to revision fix_1060) to c:\users\mahinour elsarky\appdata\local\temp\pip-req-build-a5msne2r
  Resolved https://github.com/Zeecka/pytube to commit 8d2e9a78ad6d7f72982fc9a6bc8a91b341b35dda
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/Zeecka/pytube 'C:\Users\Mahinour Elsarky\AppData\Local\Temp\pip-req-build-a5msne2r'
  Running command git checkout -b fix_1060 --track origin/fix_1060
  branch 'fix_1060' set up to track 'origin/fix_1060'.
  Switched to a new branch 'fix_1060'

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
# youtube_loader_1 = YoutubeLoader.from_youtube_url("https://youtu.be/5p248yoa3oE?si=TATgA2GMtcQ_MjEA")
# youtube_loader_2 = YoutubeLoader.from_youtube_url("https://youtu.be/kBbv2_9vcC0?si=EF6C-MNS7Ppc4TNQ")
# youtube_loaders=[]
# youtube_loaders.append(youtube_loader_1)
# youtube_loaders.append(youtube_loader_2)

# transcript_1 = youtube_loader_1.load()
# # video_info= youtube_loader._get_video_info()
# # video_publish_date= video_info["publish_date"]
# transcript_2 = youtube_loader_2.load()

# transcripts= []
# transcripts.append(transcript_1)
# transcripts.append(transcript_2)

# "https://www.youtube.com/watch?v=5p248yoa3oE",
#"https://youtu.be/kBbv2_9vcC0?si=EF6C-MNS7Ppc4TNQ"
video_urls = [
   "https://youtu.be/8UVHTgY_IVQ?si=Z19cRiDsU4_rPw1G"
 
]
youtube_loaders = [YoutubeLoader.from_youtube_url(url) for url in video_urls]
transcripts = [loader.load() for loader in youtube_loaders]


for youtube_loader in youtube_loaders:
   youtube_loader.add_video_info= True
   print(youtube_loader.add_video_info)





first_vid=transcripts[0]

yt = YouTube(f"https://www.youtube.com/watch?v=8UVHTgY_IVQ")
print(yt.author)
video_info = {
            "description": yt.description or "Unknown",
            "publish_date": yt.publish_date.strftime("%Y-%m-%d %H:%M:%S")
            if yt.publish_date
            else "Unknown",
            "author": yt.author or "Unknown",
        }
print(video_info)

# try:
#     video_info = youtube_loaders[0]._get_video_info()
# except HTTPError as e:
#     print(f"Error: {e.code} - {e.msg}")
#video_publish_date= video_info["publish_date"]

# adding one doc to the list so output is List[Document] 
# we loop in each sublist of transcripts which is list[List[Document]] and loop on each doc in that sublist
flat_transcripts = [doc for sublist in transcripts for doc in sublist]


text_splitter = RecursiveCharacterTextSplitter(separators=["\n", " "], chunk_size=10000, chunk_overlap=2200)

docs= text_splitter.split_documents(flat_transcripts)


print (f"You have {len(docs)} docs. First doc is {llm3.get_num_tokens(docs[0].page_content)} tokens")

docs[0].metadata['source']
#metadata={'source': '5p248yoa3oE'})
#https://www.youtube.com/watch?v=5p248yoa3oE


True
unknown
{'description': 'Unknown', 'publish_date': '2023-09-08 00:00:00', 'author': 'unknown'}
You have 4 docs. First doc is 1993 tokens


'8UVHTgY_IVQ'

In [28]:


template="""
You are a helpful assistant that helps retrieve distinct topics talked about in youtube videos transcripts
- Your goal is to extract the topic names,  brief 1-sentence description of the topic
- Topics can include:
- AI tools
- GPT Models
- Google Models
- LLMs
- llama Models
- Falcon Models
- Programming Languages
- AI recent News
- AI tutorials
- OpenAI
- AI for business 
- AI for education
- AI for medicine 
- AI for art and music
- Deep Learning
- NLP
- Machine Learning
- Data science
- Opportunities in AI
- AI frameworks
- Future AI
- Langchain

- Provide a brief description of the topics after the topic name, then Provide the source of the topic. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the youtube video
- ALWAYS include Brief Description beside the Topic. Example: 'Topic: Brief Description'
- Do not respond with numbers, just bullet points of all topics listed under each other. Example:
Topics:
    - Topic 1 title: topic 1 description
    - Topic 2 title: topic 2 description 
    - Topic 3 title: topic 3 description
    
- Ignore topics on policy and regulations
- Do not respond with anything outside of the transcript. If you can't extract any topics at all in the whole transcript, say 'Sorry, No topics found in the given content'
- Only pull topics from the transcript. Do not use the examples
- If the speakers names were mentioned in the transcript, instead of saying 'The speaker' refer to the names.
- Make your titles descriptive but concise. Example: 'Shaan's Experience at Twitch' should be 'Shaan's Interesting Projects At Twitch'
- A topic should be substantial, more than just a one-off comment

"""
#- Do not respond with anything outside of the transcript. If you don't see any topics, say, 'No Topics'
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_map = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

In [29]:
# % START OF EXAMPLES
# - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
# - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
# % END OF EXAMPLES

template="""
You are a helpful assistant that helps retrieve topics talked about in a youtube transcript
- You will be given a series of bullet topics of topics found
- Your goal is to exract the topic names and brief 1-sentence description of the topic
- Do not respond with numbers, just bullet points of all topics listed under each other.
- Deduplicate any bullet points you see
- If you think two or more topics are similar and can be merged, merge them together with one topic title and create a new description that fits the merged topics
- Only pull topics from the transcript. Do not use the examples.
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_combine = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])



In [30]:
chain = load_summarize_chain(llm3,
                             chain_type="map_reduce",
                             map_prompt=chat_prompt_map,
                             combine_prompt=chat_prompt_combine,
                            verbose=True
                            )

In [31]:

topics_found = chain.run({"input_documents": docs})



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant that helps retrieve distinct topics talked about in youtube videos transcripts
- Your goal is to extract the topic names,  brief 1-sentence description of the topic
- Topics can include:
- AI tools
- GPT Models
- Google Models
- LLMs
- llama Models
- Falcon Models
- Programming Languages
- AI recent News
- AI tutorials
- OpenAI
- AI for business 
- AI for education
- AI for medicine 
- AI for art and music
- Deep Learning
- NLP
- Machine Learning
- Data science
- Opportunities in AI
- AI frameworks
- Future AI
- Langchain

- Provide a brief description of the topics after the topic name, then Provide the source of the topic. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the youtube video
- ALWAYS include Brief Description beside the Topic. Example: 'Topic: Brief Description'
- Do not respond with nu

In [32]:
print(topics_found)

Topics:
- AI updates in terms of service for X Corp: X Corp updated their terms of service to include the ability to train any data on AI.
- Zoom AI companion: Zoom rebranded Zoom IQ to The Zoom AI companion, which allows users to ask questions and reference past conversations to answer queries.
- Google duet: Google rolled out similar features to Zoom AI companion with Google duet.
- Open-source Falcon 180b: Falcon 180b was launched, making it the largest publicly available open-source large language model to date.
- Anthropics Claud Pro Plan: Anthropics Claud released a Pro Plan similar to chat GPT's Pro Plan, offering more usage, priority access, and early access to new features.
- Canva plugin for chat GPT: OpenAI introduced a Canva plugin for chat GPT plus subscribers to enhance content creation.
- OpenAI developer conference: OpenAI will be hosting their first-ever developer conference to preview new tools and exchange ideas.
- Inpaint feature: Generating better images by inpaint

In [33]:
!pip install kor


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number


In [35]:
schema = Object(
    id="topic",
    description="Topic Information",
    examples=[
        ("Generative AI: An exciting tool in AI that can generate text based on input prompts.", [{"topic title": "Generative AI"}, {"description": "An exciting tool in AI that can generate text based on input prompts."}, {"tag": "AI Tools"}]),
        ("AI in relationships: The use of AI in relationship coaching and mentoring, such as the example of an AI-powered romantic relationship coaching app.", [{"topic title": "AI in relationships"}, {"description": "The use of AI in relationship coaching and mentoring, such as the example of an AI-powered romantic relationship coaching app."}, {"tag": "AI Applications"}]),
        ("AI in different industries: Exploring the potential of AI in various industries beyond consumer software and the internet.", [{"topic title": "AI in different industries"}, {"description": "Exploring the potential of AI in various industries beyond consumer software and the internet."}, {"tag": "AI Opportunities"}]),
        ("Supervised Learning: A technique in AI that is good at labeling things or computing input to outputs.", [{"topic title": "Supervised Learning"}, {"description": "A technique in AI that is good at labeling things or computing input to outputs."}, {"tag": "AI Tools"}]),
        ("Large Language Models: The power and potential of large language models in AI applications.", [{"topic title": "Large Language Models"}, {"description": "The power and potential of large language models in AI applications."}, {"tag": "AI LLMs"}]),
        ("Future growth of AI technologies: The prediction that supervised learning and generative AI will continue to grow in value and adoption over the next three years, with the potential for even greater expansion in the long term.", [{"topic title": "Future growth of AI technologies:"}, {"description": "The prediction that supervised learning and generative AI will continue to grow in value and adoption over the next three years, with the potential for even greater expansion in the long term."}, {"tag": "Future of AI"}]),
        ("Large Language Models: Dr. Andrew explains how large language models, like GPT, are built using supervised learning to predict the next word, enabling applications that can be built faster and more efficiently.", [{"topic title": "Large Language Models:"}, {"description": " Dr. Andrew explains how large language models, like GPT, are built using supervised learning to predict the next word, enabling applications that can be built faster and more efficiently."}, {"tag": "AI LLMs"}]),
 
    ],
    attributes=[
         Text(
            id="title",
            description="The title of the topic listed",
        ),
        Text(
            id="description",
            description="The description of the topic listed",
        ),
        Text(
            id="tag",
            description="The type of content being described",
        )
    ],
    many=True,
)

In [36]:
#chain = create_extraction_chain(schema, llm3)
chain = create_extraction_chain(llm3, schema)

In [37]:
topics_structured = chain.run(topics_found)

In [38]:
topics_structured

{'data': {'topic': [{'title': 'AI updates in terms of service for X Corp',
    'description': 'X Corp updated their terms of service to include the ability to train any data on AI.',
    'tag': 'AI Updates'},
   {'title': 'Zoom AI companion',
    'description': 'Zoom rebranded Zoom IQ to The Zoom AI companion, which allows users to ask questions and reference past conversations to answer queries.',
    'tag': 'AI Applications'},
   {'title': 'Google duet',
    'description': 'Google rolled out similar features to Zoom AI companion with Google duet.',
    'tag': 'AI Applications'},
   {'title': 'Open-source Falcon 180b',
    'description': 'Falcon 180b was launched, making it the largest publicly available open-source large language model to date.',
    'tag': 'AI LLMs'},
   {'title': 'Anthropics Claud Pro Plan',
    'description': "Anthropics Claud released a Pro Plan similar to chat GPT's Pro Plan, offering more usage, priority access, and early access to new features.",
    'tag': 'A

# 2. Expanding on the topics found


In [39]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n", " "], chunk_size=4000, chunk_overlap=800)

docs= text_splitter.split_documents(flat_transcripts)

In [40]:
openai_embeddings = OpenAIEmbeddings(show_progress_bar=True, embedding_ctx_length=1024)

In [41]:
!pip install faiss-cpu


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
youtube_db = FAISS.from_documents(docs, openai_embeddings)

c:\Users\Mahinour Elsarky\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:01<00:00,  1.83s/it]


In [43]:
# The system instructions. Notice the 'context' placeholder down below. This is where our relevant docs will go.
# The 'question' in the human message below won't be a question per se, but rather a topic we want to get relevant information on
system_template = """
You will be given text from youtube transcripts which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
{context}"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]

# This will pull the two messages together and get them ready to be sent to the LLM through the retriever
CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [44]:
from langchain.chains import RetrievalQA

In [45]:

# I'm also setting k=4 so the number of relevant docs we get back is 4. 
qa = RetrievalQA.from_chain_type(llm=llm3,
                                 chain_type="stuff",
                                 retriever=youtube_db.as_retriever(k=4),
                                 chain_type_kwargs = {
                                     'verbose': True,
                                     'prompt': CHAT_PROMPT
                                 })

In [46]:
topics_structured= topics_structured["data"]["topic"]

In [47]:
topics_structured

[{'title': 'AI updates in terms of service for X Corp',
  'description': 'X Corp updated their terms of service to include the ability to train any data on AI.',
  'tag': 'AI Updates'},
 {'title': 'Zoom AI companion',
  'description': 'Zoom rebranded Zoom IQ to The Zoom AI companion, which allows users to ask questions and reference past conversations to answer queries.',
  'tag': 'AI Applications'},
 {'title': 'Google duet',
  'description': 'Google rolled out similar features to Zoom AI companion with Google duet.',
  'tag': 'AI Applications'},
 {'title': 'Open-source Falcon 180b',
  'description': 'Falcon 180b was launched, making it the largest publicly available open-source large language model to date.',
  'tag': 'AI LLMs'},
 {'title': 'Anthropics Claud Pro Plan',
  'description': "Anthropics Claud released a Pro Plan similar to chat GPT's Pro Plan, offering more usage, priority access, and early access to new features.",
  'tag': 'AI Applications'},
 {'title': 'Canva plugin for 

In [48]:
for topic in topics_structured:
    query = f"""
        {topic['title']}: {topic['description']}
    """
    expanded_topic = qa.run(query)
    print(f"{topic['title']}: {topic['description']}")
    print ("\n\n")
    print(expanded_topic)
    print ("\n\n")

100%|██████████| 1/1 [00:01<00:00,  1.16s/it]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from youtube transcripts which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
there's been a handful of exciting things that have happened in the AI world this week so let's go ahead and break them down starting with the fact that X formerly known as Twitter updated their terms of service to include the ability to train any of your data on AI so they actually updated their terms of service under 2.1 operate improve and personalize our services to say we may use the information we collect and publicly available information to help train our machine learning or artificial intelligence models for the pur

100%|██████████| 1/1 [00:00<00:00,  1.45it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from youtube transcripts which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
there's been a handful of exciting things that have happened in the AI world this week so let's go ahead and break them down starting with the fact that X formerly known as Twitter updated their terms of service to include the ability to train any of your data on AI so they actually updated their terms of service under 2.1 operate improve and personalize our services to say we may use the information we collect and publicly available information to help train our machine learning or artificial intelligence models for the pur

100%|██████████| 1/1 [00:00<00:00,  2.72it/s]


...



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from youtube transcripts which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
about to dramatically change the world not necessarily in the negative positive sense in the sense that we just need to rethink everything we need to rethink how we see labor and work and what the humans roles on the Planet are so three completely different thought processes around Ai and hard to say which one I sort of lead to I definitely think there's probably bits and pieces of all three of them that I could relate to so I just love going down these rabbit holes and hearing what others think about it and getting my own t

100%|██████████| 1/1 [00:00<00:00,  1.26it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from youtube transcripts which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
there's been a handful of exciting things that have happened in the AI world this week so let's go ahead and break them down starting with the fact that X formerly known as Twitter updated their terms of service to include the ability to train any of your data on AI so they actually updated their terms of service under 2.1 operate improve and personalize our services to say we may use the information we collect and publicly available information to help train our machine learning or artificial intelligence models for the pur

100%|██████████| 1/1 [00:00<00:00,  1.19it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from youtube transcripts which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
and is generating a unique script based on what it learned all right now we got a script hey there Tech enthusiasts welcome back to another exciting video on my channel in today's episode we're diving deep into the fascinating world of artificial intelligence why am I reading this myself Let's Have Sam read it instead hey there Tech enthusiasts welcome back to another exciting video on my channel in today's episode we're diving deep into the fascinating world of artificial intelligence not bad not only do I have a script for